# Week 10 — Day 6: Hyperparameter Tuning (Random Forest)

### Imports and Load

In [1]:
import joblib
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import (confusion_matrix, precision_score, recall_score, f1_score, average_precision_score)

In [2]:
ARTIFACTS_DIR = Path("..") / "models"
REPORTS_DIR = Path("..") / "reports"
REPORTS_DIR.mkdir(exist_ok=True)

X_train, X_test, y_train, y_test = joblib.load(ARTIFACTS_DIR / "split_v1.joblib")
print("Train:", X_train.shape, "Test:", X_test.shape)

Train: (227845, 30) Test: (56962, 30)


**Feature Engineering Function**

In [3]:
def add_features(df):
    df = df.copy()
    df["log_amount"] = np.log1p(df["Amount"])
    df["hour"] = (df["Time"] // 3600).astype(int)
    df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
    df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)
    return df

In [4]:
X_train_fe = add_features(X_train)
X_test_fe = add_features(X_test)

### Setup RandomizedSearch

In [5]:
# base random forest model
rf_base = RandomForestClassifier(
    random_state=42,
    n_jobs=-1,
    class_weight="balanced_subsample"
)

**Parameter Search Space**

In [6]:
param_dist = {
    "n_estimators": [200, 300, 500],
    "max_depth": [None, 8, 12, 16],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", 0.5, 0.8]
}

In [7]:
# cross validation setup
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

**RandomizedSearchCV**

In [8]:
search = RandomizedSearchCV(
    estimator=rf_base,
    param_distributions=param_dist,
    n_iter=8,                  
    scoring="average_precision",# PR-AUC scorer
    cv=cv,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

search.fit(X_train_fe, y_train)
print("Best PR-AUC (CV):", search.best_score_)
print("Best params:", search.best_params_)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best PR-AUC (CV): 0.8386114590506754
Best params: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': None}


### Evaluate Tuned Model on Test Set

In [9]:
# finding best model
best_rf = search.best_estimator_

In [10]:
# evaluation and metrics
y_pred = best_rf.predict(X_test_fe)
y_prob = best_rf.predict_proba(X_test_fe)[:, 1]

cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
pr_auc = average_precision_score(y_test, y_prob)

print("Confusion matrix (tn, fp, fn, tp):", tn, fp, fn, tp)
print(f"Precision: {precision:.6f}")
print(f"Recall:    {recall:.6f}")
print(f"F1:        {f1:.6f}")
print(f"PR-AUC:    {pr_auc:.6f}")

Confusion matrix (tn, fp, fn, tp): 56860 4 21 77
Precision: 0.950617
Recall:    0.785714
F1:        0.860335
PR-AUC:    0.864456


### Comparison Table

In [11]:
rf_current = {"model":"RF + FE (baseline)", "precision":0.960526, "recall":0.744898, "f1":0.839080, "pr_auc":0.865561}
rf_tuned = {"model":"RF + FE (tuned)", "precision":precision, "recall":recall, "f1":f1, "pr_auc":pr_auc}

compare_df = pd.DataFrame([rf_current, rf_tuned])
compare_df

,model,precision,recall,f1,pr_auc
0,RF + FE (baseline),0.960526,0.744898,0.839080,0.865561
1,RF + FE (tuned),0.950617,0.785714,0.860335,0.864456


### Save best tuned model + search results

In [12]:
joblib.dump(best_rf, ARTIFACTS_DIR / "rf_fe_tuned_v1.joblib")
joblib.dump(search.best_params_, ARTIFACTS_DIR / "rf_fe_tuned_params_v1.joblib")

compare_df.to_csv(REPORTS_DIR / "week10_day6_rf_tuning_comparison.csv", index=False)

# Save full CV results (optional but nice)
cv_results = pd.DataFrame(search.cv_results_).sort_values("rank_test_score")
cv_results.to_csv(REPORTS_DIR / "week10_day6_rf_random_search_results.csv", index=False)

print("Saved tuned model + params + results.")

Saved tuned model + params + results.
